## Import necessary libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## Imput Foursquare credentials and Define version

In [2]:
CLIENT_ID = 'MUBDNMCRDRSEWGAQ24L4MM2VKRBKG2FSVTA5IAWFFZTTF2MC' 
CLIENT_SECRET = 'P1QHUV1VCTKILM3NXJD440VQYJSASDHETCJCRSPWGUM2DYLM' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MUBDNMCRDRSEWGAQ24L4MM2VKRBKG2FSVTA5IAWFFZTTF2MC
CLIENT_SECRET:P1QHUV1VCTKILM3NXJD440VQYJSASDHETCJCRSPWGUM2DYLM


## Assuming we are at Austin China Town, let's convert its address to latitude and longitude corrdinates

In [3]:
address = '10901 N Lamar Blvd, Austin, TX'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

30.378299 -97.687731


## 1 Searching for a specific Venue Category

In [4]:
search_query = 'Chinese'
radius = 500

### define the URL and get the request

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7ddf9a4434b957cf0c6161'},
 'response': {'venues': [{'id': '4b5a5d52f964a52041c028e3',
    'name': 'First Chinese BBQ',
    'location': {'address': '10901 N Lamar Blvd',
     'lat': 30.37844468752835,
     'lng': -97.68671173887614,
     'labeledLatLngs': [{'label': 'display',
       'lat': 30.37844468752835,
       'lng': -97.68671173887614}],
     'distance': 99,
     'postalCode': '78753',
     'cc': 'US',
     'city': 'Austin',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['10901 N Lamar Blvd',
      'Austin, TX 78753',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '60021004'},
    'referralId': 'v-1551753114',
    'hasPe

## Transform json format into a pandas dataframe

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b5a5d52f964a52041c028e3,10901 N Lamar Blvd,US,Austin,United States,99,"[10901 N Lamar Blvd, Austin, TX 78753, United ...","[{'label': 'display', 'lat': 30.37844468752835...",30.378445,-97.686712,78753,TX,First Chinese BBQ,v-1551753114,60021004
1,"[{'id': '52af3a7c3cf9994f4e043bed', 'name': 'C...",False,5468ecfe498e87b191079be1,10901 N Lamar Blvd,US,Austin,United States,61,"[10901 N Lamar Blvd, Austin, TX 78753, United ...","[{'label': 'display', 'lat': 30.37784779038532...",30.377848,-97.688096,78753,TX,New Fortune Chinese Seafood Restaurant,v-1551753114,NaN
2,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,4e9cdd740aaf015a05c3dc1b,10901 N Lamar Blvd Ste A101,US,Austin,United States,8,"[10901 N Lamar Blvd Ste A101, Austin, TX 78753...","[{'label': 'display', 'lat': 30.37824730615741...",30.378247,-97.687672,78753,TX,Duy Vietnamese Restaurant,v-1551753114,NaN


## Filter dataframe

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,First Chinese BBQ,Chinese Restaurant,10901 N Lamar Blvd,US,Austin,United States,99,"[10901 N Lamar Blvd, Austin, TX 78753, United ...","[{'label': 'display', 'lat': 30.37844468752835...",30.378445,-97.686712,78753,TX,4b5a5d52f964a52041c028e3
1,New Fortune Chinese Seafood Restaurant,Cantonese Restaurant,10901 N Lamar Blvd,US,Austin,United States,61,"[10901 N Lamar Blvd, Austin, TX 78753, United ...","[{'label': 'display', 'lat': 30.37784779038532...",30.377848,-97.688096,78753,TX,5468ecfe498e87b191079be1
2,Duy Vietnamese Restaurant,Vietnamese Restaurant,10901 N Lamar Blvd Ste A101,US,Austin,United States,8,"[10901 N Lamar Blvd Ste A101, Austin, TX 78753...","[{'label': 'display', 'lat': 30.37824730615741...",30.378247,-97.687672,78753,TX,4e9cdd740aaf015a05c3dc1b


## Visiualize Chinese restaurants near by

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='China town',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore a given venue - First Chinese BBQ

In [9]:
venue_id = '4b5a5d52f964a52041c028e3' # ID of First Chinese BBQ
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'description', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b5a5d52f964a52041c028e3',
 'name': 'First Chinese BBQ',
 'contact': {'phone': '5128358889',
  'formattedPhone': '(512) 835-8889',
  'twitter': 'firstchinesebb1'},
 'location': {'address': '10901 N Lamar Blvd',
  'lat': 30.37844468752835,
  'lng': -97.68671173887614,
  'labeledLatLngs': [{'label': 'display',
    'lat': 30.37844468752835,
    'lng': -97.68671173887614}],
  'postalCode': '78753',
  'cc': 'US',
  'city': 'Austin',
  'state': 'TX',
  'country': 'United States',
  'formattedAddress': ['10901 N Lamar Blvd',
   'Austin, TX 78753',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/first-chinese-bbq/4b5a5d52f964a52041c028e3',
 'categories': [{'id': '4bf58dd8d48988d145941735',
   'name': 'Chinese Restaurant',
   'pluralName': 'Chinese Restaurants',
   'shortName': 'Chinese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d142941735',
   'name': 'Asian Restauran

### Get this venue's overall rating

In [10]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.2


This rating is good. Let's try the other two restaurants

In [11]:
venue_id = '4e9cdd740aaf015a05c3dc1b' # ID of Duy Vietnamese Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


In [12]:
venue_id = '5468ecfe498e87b191079be1' # ID of New Fortune Chinese Seafood Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.6


This New Fortune has the highest rating. Let's explore it further

In [15]:
result['response']['venue']['tips']['count']

15

### Get the venue's tips

In [14]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7de3b34c1f6762dcc57d73'},
 'response': {'tips': {'count': 15,
   'items': [{'id': '5af04654c47cf9002c675937',
     'createdAt': 1525696084,
     'text': 'Dim sum on the weekend is well run with no long waits in between carts. More dumpling options than other places in town. Always get Chinese broccoli. Always save room for sesame balls and mochi.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5af04654c47cf9002c675937',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'lastVoteText': 'Upvoted 2 weeks ago',
     'lastUpvoteTimestamp': 1550428494,
     'todo': {'count': 0},
     'user': {'id': '1130338',
      'firstName': 'Luke',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/1130338-NMG3TYJCNPXNFJCY.jpg'}}}]}}}

In [16]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user'])

### Format column width and display all tips

In [17]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Dim sum on the weekend is well run with no long waits in between carts. More dumpling options than other places in town. Always get Chinese broccoli. Always save room for sesame balls and mochi.,5,0,5af04654c47cf9002c675937,Luke,NaN,male,1130338


In [18]:
user_id = '1130338' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [20]:
print('First Name: ' + user_data['firstName'])
print('Home City: ' + user_data['homeCity'])

First Name: Luke
Home City: Brooklyn, NY


In [21]:
user_data['tips']

{'count': 791}

This is definitely a foodie guy!

In [22]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,A true Brooklyn classic for southern Italian and good service. So many options and huge portions. Ask for recommendations and you’ll be well taken care of,0,0,5c60828d25fb7b002ca40c8b
1,Awesome mom and pop shop with fair prices and fast turnaround,0,0,5c3ddd476336be002b9ffe3e
2,That’s a fine cup of juice,0,0,5c2be9386a5950002c33694b
3,"If you are really, really into tea, or maybe want to convince someone you are, then this place is worth checking out.",0,0,5c2be914b9ac38002c42d5db
4,"There’s extra seating in a separate section, if there are no spots by the bar or buffet",0,0,5c27d39bd8096e002cfaceeb
5,Cute place with good energy and a lot of heavy duty pastas. The duck confit salad is a sleeper hit,0,0,5c27d34f9b0473002c2a44cd
6,"Enchiladas de la Casa, margarita, and repeat",0,0,5c27d30bdb2aeb002c089ef7
7,"A fusion take that doesn’t suck at all. The classic izakaya stuff is well done, but their original creations are really special. Don’t forget the brisket",0,0,5c27d2d4d8096e002cfa887c
8,Tiny lil dive with a big back yard. Nice place to take a deep breath and drink a beer,0,0,5c27d24b9b0473002c29eb6d
9,Neighborhood spot that focuses on big amounts of grilled meat,0,0,5c27d20d446ea6002c712a97


In [23]:
tip_id = '5af04654c47cf9002c675937' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

New Fortune Chinese Seafood Restaurant
{'address': '10901 N Lamar Blvd', 'lat': 30.37784779038532, 'lng': -97.68809553907404, 'labeledLatLngs': [{'label': 'display', 'lat': 30.37784779038532, 'lng': -97.68809553907404}], 'postalCode': '78753', 'cc': 'US', 'city': 'Austin', 'state': 'TX', 'country': 'United States', 'formattedAddress': ['10901 N Lamar Blvd', 'Austin, TX 78753', 'United States']}


In [24]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.prefix,photo.suffix,superuser,tips.count
0,,Laura,female,,152222593,Brown,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/152222593-SLJ2XRS3GHORBCWO.jpg,NaN,0
1,,Zak,none,"Boulder, CO",151539825,Loyd,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/151539825-W3INMQ5GFMI1LV0L.jpg,NaN,0
2,Enthusiast.,CJN,male,"New York, New York",3722,NaN,"[{'type': 'created', 'count': 8, 'items': []}]",https://fastly.4sqi.net/img/user/,/3722-EWAITHKTWQSRF0ET.jpg,1.0,112
3,,Cecily,female,"Austin, Texas",147904320,Gold,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/147904320-NAJF4A25FNRWF4HV.jpg,NaN,0
4,,Kiana,female,"New York, NY",12888345,Pirouz,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/12888345-TVSNXUEKSRLARQ52.jpg,NaN,0
5,Twitter: @NicholasLFuller\nInsta: @NicholasFuller,Nicholas,male,"New York, NY",1955596,Fuller,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/1955596_vHIdRAq7_zocD0-znvaVEgrFm8PdX_PzgV5vtnf3EnO_N3hdIMdd-P9BHv6eJfYsBGN-5qaVh.jpg,NaN,18
6,"Musician, Booker, Recording Engineer, Aesthetic, Cat Vs. Dogs www.facebook.com/ShowersTheBand",Marshall,male,"The Woodlands, TX",9222629,Walker,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/CJ05OQZQUSU3ROSM.jpg,NaN,13
7,,Tom,male,"Melbourne, Victoria, Australia",85994956,Hyde,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/85994956-A3CRIECJQMNVWC2I.jpg,NaN,0
8,,Evgeny,male,"Brooklyn, NY",678652,Nitsenko,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/678652-Z10VMSTGKRK0U03K.jpg,NaN,1
9,,Jeff,male,"Portland, OR",18759,Simmons,"[{'type': 'created', 'count': 4, 'items': []}]",https://fastly.4sqi.net/img/user/,/18759-EFH5CLZRHJBFTLFS.jpg,NaN,4


In [26]:
user_data

{'id': '1130338',
 'firstName': 'Luke',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/simiangibbons',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/1130338-NMG3TYJCNPXNFJCY.jpg'},
 'friends': {'count': 78,
  'groups': [{'type': 'others',
    'name': 'Other friends',
    'count': 78,
    'items': [{'id': '152222593',
      'firstName': 'Laura',
      'lastName': 'Brown',
      'gender': 'female',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/152222593-SLJ2XRS3GHORBCWO.jpg'},
      'tips': {'count': 0},
      'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
      'homeCity': '',
      'bio': '',
      'contact': {}},
     {'id': '151539825',
      'firstName': 'Zak',
      'lastName': 'Loyd',
      'gender': 'none',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/151539825-W3INMQ5GFMI1LV0L.jpg'},
      'tips': {'count': 0},
      'lists': {'groups': [{'type': 

In [27]:
Image(url='https://fastly.4sqi.net/img/user/300x300/1130338-NMG3TYJCNPXNFJCY.jpg')

## Explore locations near this Chinese Restaurant

In [28]:
latitude = 30.377848
longtitude = -97.688096
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
import requests
results = requests.get(url).json()
'There are {} around New Fortune Chinese Seafood Restaurant.'.format(len(results['response']['groups'][0]['items']))


'There are 29 around New Fortune Chinese Seafood Restaurant.'

### Get relevant part of JSON

In [29]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4a5cbc97f964a5207bbc1fe3',
  'name': 'Pho Saigon',
  'location': {'address': '10901 N Lamar Blvd #108',
   'crossStreet': 'Chinatown Center',
   'lat': 30.378036303841842,
   'lng': -97.68720454742908,
   'labeledLatLngs': [{'label': 'display',
     'lat': 30.378036303841842,
     'lng': -97.68720454742908}],
   'distance': 54,
   'postalCode': '78753',
   'cc': 'US',
   'city': 'Austin',
   'state': 'TX',
   'country': 'United States',
   'formattedAddress': ['10901 N Lamar Blvd #108 (Chinatown Center)',
    'Austin, TX 78753',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d14a941735',
    'name': 'Vietnamese Restaurant',
    'pluralName': 'Vietnamese Restaurants',
    'shortName': 'Vietnamese',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vietnamese_',
     'suffix': '.png'},
    'prima

### convert Json to a clean dataframe

In [30]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pho Saigon,Vietnamese Restaurant,10901 N Lamar Blvd #108,US,Austin,United States,Chinatown Center,54,"[10901 N Lamar Blvd #108 (Chinatown Center), Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.378036303841842, 'lng': -97.68720454742908}]",30.378036,-97.687205,NaN,78753,TX,4a5cbc97f964a5207bbc1fe3
1,New Fortune Chinese Seafood Restaurant,Cantonese Restaurant,10901 N Lamar Blvd,US,Austin,United States,NaN,35,"[10901 N Lamar Blvd, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.37784779038532, 'lng': -97.68809553907404}]",30.377848,-97.688096,NaN,78753,TX,5468ecfe498e87b191079be1
2,Baguette House & Cafe,Vietnamese Restaurant,10901 N Lamar Blvd,US,Austin,United States,Ferguson Dr.,117,"[10901 N Lamar Blvd (Ferguson Dr.), Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.377137365679467, 'lng': -97.68682001913672}]",30.377137,-97.686820,NaN,78753,TX,4b54b3b5f964a520e6c727e3
3,Korean Grill,Korean Restaurant,10901 N Lamar Blvd #204,US,Austin,United States,NaN,147,"[10901 N Lamar Blvd #204, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.378983095137333, 'lng': -97.68694230024873}]",30.378983,-97.686942,NaN,78753,TX,4c379ff7ae2da593103bffc5
4,Foot Relax,Massage Studio,10901 N Lamar Blvd,US,Austin,United States,NaN,116,"[10901 N Lamar Blvd, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.377823130009208, 'lng': -97.6865157466896}]",30.377823,-97.686516,NaN,78753,TX,4e29e7b7315185636a0872e9
5,First Chinese BBQ,Chinese Restaurant,10901 N Lamar Blvd,US,Austin,United States,NaN,118,"[10901 N Lamar Blvd, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.37844468752835, 'lng': -97.68671173887614}]",30.378445,-97.686712,NaN,78753,TX,4b5a5d52f964a52041c028e3
6,MT Supermarket,Supermarket,10901 N Lamar Blvd,US,Austin,United States,Kramer Ln,130,"[10901 N Lamar Blvd (Kramer Ln), Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.378199791324707, 'lng': -97.68642971895694}]",30.378200,-97.686430,NaN,78753,TX,4a7e282ef964a52037f11fe3
7,Bambū Desserts & Drinks,Dessert Shop,10901 N Lamar Blvd #A120,US,Austin,United States,NaN,71,"[10901 N Lamar Blvd #A120, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.377389680129244, 'lng': -97.6872132724749}]",30.377390,-97.687213,NaN,78753,TX,539a4771498efdb7bfa5fe66
8,Chinatown Austin,Convenience Store,10901 N Lamar Blvd,US,Austin,United States,NaN,90,"[10901 N Lamar Blvd, Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.378431103588724, 'lng': -97.68708172176044}]",30.378431,-97.687082,North Lamar,78753,TX,509597a2e4b0a70262d28525
9,Chuy's,Mexican Restaurant,10520 N Lamar Blvd,US,Austin,United States,at McPhaul Street,412,"[10520 N Lamar Blvd (at McPhaul Street), Austin, TX 78753, United States]","[{'label': 'display', 'lat': 30.37508014595138, 'lng': -97.6905871670896}]",30.375080,-97.690587,NaN,78753,TX,41293380f964a520670c1fe3


### Visualize these locations

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around New Fortune


# add New Fortune as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Fortune',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Explore trending venues near by

In [32]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7de967dd57977c0476f9a2'},
 'response': {'venues': []}}

In [33]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [34]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

It seems that no popular places near this restaurant. So if visotrs want to find some places for fun, they may need to go to downtown.